# Lab 2 Assignment

**__Name:__ Mit Dani**


In the assignment, you will analyze which factors are associated with the number of likes a Facebook post gets. The dataset is from https://archive.ics.uci.edu/ml/datasets/Facebook+metrics. All posts are from the page of a renowned cosmetics brand.

The dependent variable is `like`.

In [56]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt

In [57]:
fb_like = pd.read_csv('http://fengmai.net/download/data/bia652/dataset_Facebook.csv', sep=";")
# remove unneeded variables
not_needed = ['Page total likes', 'Total Interactions']
fb_like = fb_like.drop(not_needed, axis = 1)

In [58]:
fb_like.head(5)


,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share
0,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0
1,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0
2,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0
3,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0
4,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0


## Q1 - Modeling with categorical Xs

- Use [pd.get_dummies()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) to create dummy variables for the following variables: `Type`, `Category`, `Post Month`, `Post Weekday`, `Post Hour`. Make sure that `drop_firt=True` to avoid multicollinearity problem.
- You may want to read [this post](https://stackoverflow.com/questions/24109779/running-get-dummies-on-several-dataframe-columns) to create dummies for multiple columns.
- The origninal columns will be automatically dropped after you create the dummies. 
- You may want to check if any rows have missing variables and drop them accordingly (use [dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)). 
- Create a regression model that uses all the variables (other than like) as independent variables. You'll need to use `statsmodels` and not `sklearn`. Display the OLS Regression Results.  
- No need to split the data to train-test sets for this exercise. 

In [59]:
#### YOUR CODE HERE ####
import statsmodels.api as sm
fb_like = pd.get_dummies(data=fb_like, columns= ['Type','Category','Post Month','Post Weekday', 'Post Hour'], drop_first=True)
fb_like= fb_like.dropna(axis = 0)
fb_like

,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Type_Photo,Type_Status,Type_Video,Category_2,Category_3,Post Month_2,Post Month_3,Post Month_4,Post Month_5,Post Month_6,Post Month_7,Post Month_8,Post Month_9,Post Month_10,Post Month_11,Post Month_12,Post Weekday_2,Post Weekday_3,Post Weekday_4,Post Weekday_5,Post Weekday_6,Post Weekday_7,Post Hour_2,Post Hour_3,Post Hour_4,Post Hour_5,Post Hour_6,Post Hour_7,Post Hour_8,Post Hour_9,Post Hour_10,Post Hour_11,Post Hour_12,Post Hour_13,Post Hour_14,Post Hour_15,Post Hour_16,Post Hour_17,Post Hour_18,Post Hour_19,Post Hour_20,Post Hour_22,Post Hour_23
0,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,0.0,5400,9218,810,756,1003,5654,3230,422,10,125.0,41.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
495,0.0,4684,7536,733,708,985,4750,2876,392,5,53.0,26.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
496,0.0,3480,6229,537,508,687,3961,2104,301,0,53.0,22.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
497,0.0,3778,7216,625,572,795,4742,2388,363,4,93.0,18.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [60]:
Y = fb_like['like']
X = fb_like.drop(columns='like',axis=1)
x = sm.add_constant(X)
model = sm.OLS(Y,x)
results = model.fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                   like   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     937.5
Date:                Mon, 22 Nov 2021   Prob (F-statistic):               0.00
Time:                        02:50:44   Log-Likelihood:                -2387.3
No. Observations:                 495   AIC:                             4885.
Df Residuals:                     440   BIC:                             5116.
Df Model:                          54                                         
Covariance Type:            nonrobust                                         
                                                                          coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

# Q2 Bootstrap confidence interval

Your tasks is to use the boostrap method to construct a 95% CI for the coefficients for **comment** and **share** using the same model as in Q1. Do you get similar CI to Q1? Why or why not?

In [61]:
from tqdm.auto import tqdm
import statsmodels.formula.api as smf
bootstrapped_ests = []
for i in tqdm(range(5000)):
    # YOUR CODE HERE
    fb_like_boot = fb_like.sample(n=len(fb_like), replace=True)
    fb_like_boot = smf.ols(formula='like ~ comment + share', data=fb_like_boot).fit()
    bootstrapped_ests.append(fb_like_boot.params[['comment', 'share']])


    
b_comments, b_shares = zip(*bootstrapped_ests)
 
print("The 95% bootstrapped CI of b_comments is [{:.2f}, {:.2f}].".format(
    np.percentile(b_comments, 2.5),
    np.percentile(b_comments, 97.5)))

print("The 95% bootstrapped CI of b_shares is [{:.2f}, {:.2f}].".format(
    np.percentile(b_shares, 2.5),
    np.percentile(b_shares, 97.5)))


  0%|          | 0/5000 [00:00<?, ?it/s]

The 95% bootstrapped CI of b_comments is [2.06, 5.47].
The 95% bootstrapped CI of b_shares is [4.70, 7.00].


**The OLS confidence interval assumes a normal distribution of coefficient estimators. The bootstrap characterizes the empirical distribution of coefficient estimators (which could be non-normal). As a result, if some OLS assumptions are violated (conditional normality, homoscedasticity, and independent observations), the empirical distribution of coefficient estimators will deviate from the theoretical normal distribution, as would the confidence interval.**

## Q3 - Use AIC/BIC for model selection

- Read the [example here](https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html#sphx-glr-auto-examples-linear-model-plot-lasso-model-selection-py) and the [manual here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLarsIC.html#sklearn.linear_model.LassoLarsIC). You may want to create a separate notebook to run the example.
- Use `LassoLarsIC` to select the best model using the **`bic`** criterion. 
- There is no need to split the data to train-test sets for this exercise. 
- Which X variables are selected by the `LassoLarsIC` method? Answer using variable names. 

In [116]:
#### YOUR CODE HERE ####
from sklearn.linear_model import LassoLarsIC, LassoCV
from sklearn import linear_model
reg = linear_model.LassoLarsIC(criterion = 'bic', normalize = False)
reg.fit(X,Y)
print(reg.coef_)
X.columns[np.abs(reg.coef_) > 0]




[ 0.00000000e+00 -1.82606940e-04  0.00000000e+00  1.02995879e+00
 -9.93721391e-01  0.00000000e+00 -3.77991985e-05  3.76489083e-03
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00]


Index(['Lifetime Post Total Reach', 'Lifetime Engaged Users',
       'Lifetime Post Consumers',
       'Lifetime Post Impressions by people who have liked your Page',
       'Lifetime Post reach by people who like your Page'],
      dtype='object')

## Q4 - Compare out-of-sample perormance

*   List item
*   List item



- Split the data to 90% train and 10% test set. 
- Train three models using the train set:
   * `LassoLarsIC` model (a model that uses BIC to select variables)
   * a full model (a model that includes all X variables)
   * a `LassoCV` model (a model that uses CV on train set to select variables)
- Remember to train a scaler using train set only, and transform both X_train and X_test. 
- Compare their test-set performance using MSE. Which model has the best performance? 

In [102]:
#### YOUR CODE HERE ####
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
X_train, X_test, y_train, y_test = train_test_split(x, Y, test_size=0.1,random_state=4)

In [103]:
X_train.shape

(445, 55)

In [104]:
X_test.shape


(50, 55)

In [105]:
y_train.shape

(445,)

In [106]:
y_test.shape


(50,)

In [107]:
scaler = preprocessing.StandardScaler().fit(X_train)

In [108]:
model_Lassoirc = linear_model.LassoLarsIC(criterion = 'bic', normalize = False)
model_Lassoirc.fit(scaler.transform(X_train), y_train)

LassoLarsIC(criterion='bic', normalize=False)

In [109]:
from sklearn.linear_model import LinearRegression
model_Linear = LinearRegression()
model_Linear.fit(scaler.transform(X_train),y_train)

LinearRegression()

In [110]:
model_lassocv = LassoCV(cv=5,normalize=False)
model_lassocv.fit(scaler.transform(X_train), y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  FutureWarning,


LassoCV(cv=5, normalize=False)

In [111]:
from sklearn import metrics

In [112]:
y1_test = model_Lassoirc.predict(scaler.transform(X_test)) 
metrics.mean_squared_error(y_true= y_test, y_pred=y1_test)

15236.967541955088

In [113]:
y2_test = model_lassocv.predict(scaler.transform(X_test))
metrics.mean_squared_error(y_true=y_test, y_pred=y2_test)

5242.046771571953

In [114]:
y3_test = model_Linear.predict(scaler.transform(X_test))
metrics.mean_squared_error(y_test,y3_test)

5586.028418001769

Lasso CV offers the best model performance since it has the lowest MSE, which is the desired model performance metric here.

